## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Jalu,LY,2020-11-08 00:38:49,29.03,21.55,60.71,83,0,10.16,clear sky
1,1,Nizwa,OM,2020-11-08 00:38:49,22.93,57.53,64.90,79,0,1.16,clear sky
2,2,Kapaa,US,2020-11-08 00:38:50,22.08,-159.32,80.60,83,90,8.05,moderate rain
3,3,Vaini,TO,2020-11-08 00:38:50,-21.20,-175.20,82.40,69,20,5.82,few clouds
4,4,Sioux Lookout,CA,2020-11-08 00:37:14,50.10,-91.92,32.00,92,90,3.36,light intensity drizzle


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)



,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Kapaa,US,2020-11-08 00:38:50,22.08,-159.32,80.60,83,90,8.05,moderate rain
3,3,Vaini,TO,2020-11-08 00:38:50,-21.20,-175.20,82.40,69,20,5.82,few clouds
14,14,Saint-Georges,GD,2020-11-08 00:38:51,12.06,-61.75,80.60,88,20,14.99,few clouds
29,29,Yulara,AU,2020-11-08 00:38:55,-25.24,130.99,82.40,10,91,14.99,overcast clouds
33,33,Tabou,CI,2020-11-08 00:38:56,4.42,-7.35,78.44,87,52,5.97,broken clouds
38,38,The Valley,AI,2020-11-08 00:37:24,18.22,-63.06,81.00,78,40,11.41,scattered clouds
61,61,Sahuaripa,MX,2020-11-08 00:39:04,29.05,-109.23,75.49,56,0,11.34,clear sky
69,69,Butaritari,KI,2020-11-08 00:39:05,3.07,172.79,81.28,72,11,10.22,few clouds
71,71,Jeremie,HT,2020-11-08 00:39:05,18.65,-74.12,79.12,77,100,0.76,light rain
72,72,Ewa Beach,US,2020-11-08 00:38:26,21.32,-158.01,87.80,55,40,6.93,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Kapaa,US,2020-11-08 00:38:50,22.08,-159.32,80.60,83,90,8.05,moderate rain
3,3,Vaini,TO,2020-11-08 00:38:50,-21.20,-175.20,82.40,69,20,5.82,few clouds
14,14,Saint-Georges,GD,2020-11-08 00:38:51,12.06,-61.75,80.60,88,20,14.99,few clouds
29,29,Yulara,AU,2020-11-08 00:38:55,-25.24,130.99,82.40,10,91,14.99,overcast clouds
33,33,Tabou,CI,2020-11-08 00:38:56,4.42,-7.35,78.44,87,52,5.97,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
655,655,Mitsamiouli,KM,2020-11-08 00:35:48,-11.38,43.28,80.04,73,16,11.61,few clouds
662,662,Kalmunai,LK,2020-11-08 00:40:20,7.42,81.82,78.64,83,40,5.64,light rain
666,666,Kieta,PG,2020-11-08 00:40:21,-6.22,155.63,85.23,66,100,3.83,light rain
669,669,Marawi,PH,2020-11-08 00:40:22,8.00,124.29,81.00,76,28,5.99,scattered clouds


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Kapaa,US,80.60,moderate rain,22.08,-159.32,
3,Vaini,TO,82.40,few clouds,-21.20,-175.20,
14,Saint-Georges,GD,80.60,few clouds,12.06,-61.75,
29,Yulara,AU,82.40,overcast clouds,-25.24,130.99,
33,Tabou,CI,78.44,broken clouds,4.42,-7.35,
38,The Valley,AI,81.00,scattered clouds,18.22,-63.06,
61,Sahuaripa,MX,75.49,clear sky,29.05,-109.23,
69,Butaritari,KI,81.28,few clouds,3.07,172.79,
71,Jeremie,HT,79.12,light rain,18.65,-74.12,
72,Ewa Beach,US,87.80,scattered clouds,21.32,-158.01,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
hotel_df


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Kapaa,US,80.60,moderate rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
3,Vaini,TO,82.40,few clouds,-21.20,-175.20,Keleti Beach Resort
14,Saint-Georges,GD,80.60,few clouds,12.06,-61.75,Siesta Hotel
29,Yulara,AU,82.40,overcast clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
33,Tabou,CI,78.44,broken clouds,4.42,-7.35,hotêl doufoulougou
...,...,...,...,...,...,...,...
655,Mitsamiouli,KM,80.04,few clouds,-11.38,43.28,Foyer ADM
662,Kalmunai,LK,78.64,light rain,7.42,81.82,VS Villa
666,Kieta,PG,85.23,light rain,-6.22,155.63,
669,Marawi,PH,81.00,scattered clouds,8.00,124.29,Derogongan Residence


In [28]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace("",np.nan, inplace=True)
hotel_df.head(52)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(52)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Kapaa,US,80.60,moderate rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
3,Vaini,TO,82.40,few clouds,-21.20,-175.20,Keleti Beach Resort
14,Saint-Georges,GD,80.60,few clouds,12.06,-61.75,Siesta Hotel
29,Yulara,AU,82.40,overcast clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
33,Tabou,CI,78.44,broken clouds,4.42,-7.35,hotêl doufoulougou
38,The Valley,AI,81.00,scattered clouds,18.22,-63.06,CeBlue Villas & Beach Resort
61,Sahuaripa,MX,75.49,clear sky,29.05,-109.23,Estatua de Sahuaripa
69,Butaritari,KI,81.28,few clouds,3.07,172.79,Isles Sunset Lodge
71,Jeremie,HT,79.12,light rain,18.65,-74.12,Auberge’Inn
72,Ewa Beach,US,87.80,scattered clouds,21.32,-158.01,Hampton Inn & Suites Oahu/Kapolei


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Kapaa,US,80.60,moderate rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
3,Vaini,TO,82.40,few clouds,-21.20,-175.20,Keleti Beach Resort
14,Saint-Georges,GD,80.60,few clouds,12.06,-61.75,Siesta Hotel
29,Yulara,AU,82.40,overcast clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
33,Tabou,CI,78.44,broken clouds,4.42,-7.35,hotêl doufoulougou
...,...,...,...,...,...,...,...
653,Mount Isa,AU,91.40,scattered clouds,-20.73,139.50,ibis Styles Mt Isa Verona
655,Mitsamiouli,KM,80.04,few clouds,-11.38,43.28,Foyer ADM
662,Kalmunai,LK,78.64,light rain,7.42,81.82,VS Villa
669,Marawi,PH,81.00,scattered clouds,8.00,124.29,Derogongan Residence


In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [39]:
# # 11a. Add a marker layer for each city to the map. 
# locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
# 
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))